# Data Requests 

## Setup

In [1]:
# Import modules
import time

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


In [2]:
# API connexion
client_id = 'cec8979c027344f98b471a991aa415ad'
client_secret = 'ad69bcba55b349f98c1344b006c708bd'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Get playlists infos 

In [ ]:
# API scrapping
# Set category IDs
category_ids = [
    '0JQ5DAqbMKFNQ0fGp4byGU', '0JQ5DAqbMKFx0uLQR2okcc', '0JQ5DAqbMKFFtlLYUHv8bT', '0JQ5DAqbMKFzHmL4tf05da', '0JQ5DAqbMKFLjmiZRss79w',
    '0JQ5DAqbMKFAUsdyVjCQuL', '0JQ5DAqbMKFziKOShCi009', '0JQ5DAqbMKFQ1UFISXj59F', '0JQ5DAqbMKFSCjnQr8QZ3O', '0JQ5DAqbMKFLb2EqgLtpjC', 
    '0JQ5DAqbMKFQiK2EHwyjcU', '0JQ5DAqbMKFObNLOHydSW8', '0JQ5DAqbMKFPrEiAOxgac3', '0JQ5DAqbMKFKLfwjuJMoNC', '0JQ5DAqbMKFRY5ok2pxXJ0', 
    '0JQ5DAqbMKFHOzuVTgTizF', '0JQ5DAqbMKFP9UNkavIGrM', '0JQ5DAqbMKFIVNxQgRNSg0', '0JQ5DAqbMKFFzDl7qN9Apr', '0JQ5DAqbMKFCuoRTxhYWow', 
    '0JQ5DAqbMKFFoimhOqWzLB', '0JQ5DAqbMKFIRybaNTYXXy', '0JQ5DAqbMKFPw634sFwguI', '0JQ5DAqbMKFJw7QLnM27p6', '0JQ5DAqbMKFCbimwdOYlsl', 
    '0JQ5DAqbMKFy78wprEpAjl', '0JQ5DAqbMKFF9bY76LXmfI', '0JQ5DAqbMKFImHYGo3eTSg', '0JQ5DAqbMKFFsW9N8maB6z', '0JQ5DAqbMKFCfObibaOZbv', 
    '0JQ5DAqbMKFGnsSfvg90Wo', '0JQ5DAqbMKFQ00XGBls6ym', '0JQ5DAqbMKFCWjUTdzaG0e', '0JQ5DAqbMKFRKBHIxJ5hMm', '0JQ5DAqbMKFAJ5xb0fwo9m', 
    '0JQ5DAqbMKFGvOw3O4nLAf', '0JQ5DAqbMKFxXaXKP7zcDp', '0JQ5DAqbMKFDkd668ypn6O', '0JQ5DAqbMKFRieVZLLoo9m', '0JQ5DAqbMKFI3pNLtYMD9S', 
    '0JQ5DAqbMKFEOEBCABAxo9', '0JQ5DAqbMKFEC4WFtoNRpw', '0JQ5DAqbMKFAjfauKLOZiv', '0JQ5DAqbMKFOOxftoKZxod', '0JQ5DAqbMKFJKoGyUMo2hE', 
    '0JQ5DAqbMKFEZPnFQSFB1T', '0JQ5DAqbMKFDXXwE9BDJAr', '0JQ5DAqbMKFA6SOHvT3gck', '0JQ5DAqbMKFIpEuaCnimBj', '0JQ5DAqbMKFAXlCG6QvYQ4', 
    '0JQ5DAqbMKFDBgllo2cUIN', '0JQ5DAqbMKFQIL0AXnG5AK', '0JQ5DAqbMKFOzQeOmemkuw', '0JQ5DAqbMKFBweu2euf3Sj', '0JQ5DAqbMKFAQy4HL4XU2D'
]

# Store missing categories
missing_categories = []

# Loop over all categories and get playlists
playlists = []
for category_id in category_ids:
    try:
        # Get first 50 playlists in category
        response = sp.category_playlists(category_id, limit=50)
        category_playlists = response["playlists"]["items"]
        
        # Check if there are more playlists in category
        while response["playlists"]["next"]:
            response = sp.next(response["playlists"])
            category_playlists += response["playlists"]["items"]
        
        # Append category playlists to list
        for playlist in category_playlists:
            playlist_id = playlist["id"]
            playlist_name = playlist["name"]
            playlist_owner = playlist["owner"]["display_name"]
            playlists.append({
                "category_id": category_id, 
                "playlist_id": playlist_id, 
                "playlist_name": playlist_name,
                "playlist_owner": playlist_owner
            })
    except: 
        missing_categories.append(category_id)
        print(f"Category with ID {category_id} doesn't exist.")

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["category_id", "playlist_id", "playlist_name", "playlist_owner"])

print(df_playlists)


In [ ]:
# HTML scraping for Charts playlists
# URLs to scrape
urls = [
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1h',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1i',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1j'
]

# Set up the web driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

top_playlists = []

for url in urls:
    driver.get(url)
    time.sleep(5)  # Wait for playlists to load

    # Find playlist elements by class name
    playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
    last_playlist = playlists[-1]

    # Scroll to load all playlists
    while True:
        driver.execute_script("arguments[0].scrollIntoView();", last_playlist)
        time.sleep(2)
        new_playlists = driver.find_elements(By.CLASS_NAME, 'Box__BoxComponent-sc-y4nds-0')
        if new_playlists[-1] != last_playlist:
            last_playlist = new_playlists[-1]
            playlists = new_playlists
        else:
            break

    # Extract playlist information
    for playlist in playlists:
        # Get aria-labelledby attribute
        aria_label = playlist.get_attribute('aria-labelledby')
        
        # Extract playlist ID from aria-labelledby
        if aria_label and "spotify:playlist:" in aria_label:
            playlist_id = aria_label.split("spotify:playlist:")[1].split()[0]
            top_playlists.append({
                "category_id": '0JQ5DAudkNjCgYMM0TZXDw', 
                "playlist_id": playlist_id, 
                "playlist_owner": 'Spotify'
            })

# Quit the driver
driver.quit()

# Convert data to DataFrame and print
df_top_playlists = pd.DataFrame(top_playlists, columns=["category_id", "playlist_id", "playlist_owner"])
print(df_top_playlists)


In [ ]:
# Parse playlists info into excel
# Combine the manual and API playlists into one DataFrame
df_combined = pd.concat([df_playlists, df_top_playlists], ignore_index=True)
df_combined.to_excel("playlists_info.xlsx", index=False)

# Drop duplicates based on the "playlist_id" column, keeping the first occurrence
df_cleaned = df_combined.drop_duplicates(subset='playlist_id', keep='first')
df_cleaned.to_excel("playlists_info_clean.xlsx", index=False)


## Get tracks info

### Setup

In [3]:
# Load data
df = pd.read_excel('playlists_info_clean.xlsx')

# Remove playlists with "This is" and "Written By" in the name
df = df[~df['playlist_name'].str.contains('This Is', na=False)]
df = df[~df['playlist_name'].str.contains('Written By', na=False)]


In [4]:
# Define scraping functions
# Define a function to get all the tracks in a playlist, handling the API limit
def get_all_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Define a function to get all the audio features for a set of tracks, handling the API limit
def get_all_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        try:
            features = sp.audio_features(batch)
        except:
            continue
        audio_features.extend(features)
    return audio_features

def get_playlist_track_info(playlist_id):
    # Get the playlist info
    playlist_info = sp.playlist(playlist_id)
    if not playlist_info:
        return None
    
    playlist_name = playlist_info['name']
    playlist_followers = playlist_info['followers']['total']
    playlist_owner = playlist_info['owner']['display_name']
    
    # Get all the tracks in the playlist
    playlist_tracks = get_all_playlist_tracks(playlist_id)
    track_ids = [track['track']['id'] for track in playlist_tracks if track['track'] is not None]
    
    if not track_ids:
        return None
    
    # Get the track info in batches of 50
    track_info = []
    for i in range(0, len(track_ids), 50):
        batch_tracks = track_ids[i:i+50]
        tracks = sp.tracks(batch_tracks)['tracks']
        track_info.extend(tracks)

    if track_info is not None:
        track_names = [track['name'] for track in track_info]
        track_popularities = [track['popularity'] for track in track_info]
        track_release_dates = [track['album']['release_date'] for track in track_info]
        artist_ids = [track['artists'][0]['id'] for track in track_info]
        
        artist_info = []
        for i in range(0, len(artist_ids), 50):
            batch_artists = artist_ids[i:i+50]
            artists = sp.artists(batch_artists)['artists']
            artist_info.extend(artists)
        
        artist_names = [artist['name'] for artist in artist_info]
        artist_popularities = [artist['popularity'] for artist in artist_info]

        # Combine everything into a DataFrame
        playlist_df = pd.DataFrame({
            'playlist_name': playlist_name,
            'playlist_followers': playlist_followers,
            'playlist_owner': playlist_owner,
            'track_name': track_names,
            'track__popularity': track_popularities,
            'track_release_date': track_release_dates,
            'artist_name': artist_names,
            'artist_popularity': artist_popularities
        })
    else:
        return None

    # Get the audio features
    audio_features = get_all_audio_features(track_ids)
    if audio_features is None:
        return None
    
    # Add the audio features to the DataFrame
    try:
        audio_features_df = pd.DataFrame(audio_features)
        playlist_df = pd.concat([playlist_df, audio_features_df], axis=1)
    except:
        return None
    
    return playlist_df

# Define a function to loop over all the playlists and combine the results
def get_all_playlist_track_info(playlist_ids):
    playlist_dfs = []
    for playlist_id in playlist_ids:
        print(f"Processing playlist: {playlist_id}")
        playlist_df = get_playlist_track_info(playlist_id)
        if playlist_df is not None:
            playlist_dfs.append(playlist_df)
    all_playlist_df = pd.concat(playlist_dfs, ignore_index=True)
    return all_playlist_df


### Scrape the data for each category

In [ ]:
# Tops
category_id = "0JQ5DAudkNjCgYMM0TZXDw"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tops = get_all_playlist_track_info(playlist_ids)
df_tops.to_excel("df_tops.xlsx", index = False)


In [ ]:
# Afro
category_id = "0JQ5DAqbMKFNQ0fGp4byGU"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_pop = get_all_playlist_track_info(playlist_ids)
df_pop.to_excel("df_afro.xlsx", index = False)


In [ ]:
# A la Maison
category_id = "0JQ5DAqbMKFx0uLQR2okcc"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_maison = get_all_playlist_track_info(playlist_ids)
df_maison.to_excel("df_maison.xlsx", index = False)


In [ ]:
# Alternative
category_id = "0JQ5DAqbMKFFtlLYUHv8bT"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_alternative = get_all_playlist_track_info(playlist_ids)
df_alternative.to_excel("df_alternative.xlsx", index = False)


In [ ]:
# Ambiance
category_id = "0JQ5DAqbMKFzHmL4tf05da"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambiance = get_all_playlist_track_info(playlist_ids)
df_ambiance.to_excel("df_ambiance.xlsx", index = False)


In [ ]:
# Ambient
category_id = "0JQ5DAqbMKFLjmiZRss79w"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambient = get_all_playlist_track_info(playlist_ids)
df_ambient.to_excel("df_ambient.xlsx", index = False)


In [ ]:
# Amour
category_id = "0JQ5DAqbMKFAUsdyVjCQuL"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_amour = get_all_playlist_track_info(playlist_ids)
df_amour.to_excel("df_amour.xlsx", index = False)


In [ ]:
# Anime
category_id = "0JQ5DAqbMKFziKOShCi009"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_anime = get_all_playlist_track_info(playlist_ids)
df_anime.to_excel("df_anime.xlsx", index = False)


In [ ]:
# Arabe
category_id = "0JQ5DAqbMKFQ1UFISXj59F"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_arabe = get_all_playlist_track_info(playlist_ids)
df_arabe.to_excel("df_arabe.xlsx", index = False)


In [ ]:
# Bien etre
category_id = "0JQ5DAqbMKFLb2EqgLtpjC"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_bienetre = get_all_playlist_track_info(playlist_ids)
df_bienetre.to_excel("df_bienetre.xlsx", index = False)


In [ ]:
# Blues
category_id = "0JQ5DAqbMKFQiK2EHwyjcU"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_blues = get_all_playlist_track_info(playlist_ids)
df_blues.to_excel("df_blues.xlsx", index = False)


In [ ]:
# Caraibes
category_id = "0JQ5DAqbMKFObNLOHydSW8"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_caraibes = get_all_playlist_track_info(playlist_ids)
df_caraibes.to_excel("df_caraibes.xlsx", index = False)


In [ ]:
# Classique
category_id = "0JQ5DAqbMKFPrEiAOxgac3"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_classique = get_all_playlist_track_info(playlist_ids)
df_classique.to_excel("df_classique.xlsx", index = False)


In [ ]:
# Country
category_id = "0JQ5DAqbMKFKLfwjuJMoNC"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_country = get_all_playlist_track_info(playlist_ids)
df_country.to_excel("df_country.xlsx", index = False)


In [ ]:
# Cuisine
category_id = "0JQ5DAqbMKFRY5ok2pxXJ0"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_cuisine = get_all_playlist_track_info(playlist_ids)
df_cuisine.to_excel("df_cuisine.xlsx", index = False)


In [ ]:
# Dance
category_id = "0JQ5DAqbMKFHOzuVTgTizF"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dance = get_all_playlist_track_info(playlist_ids)
df_dance.to_excel("df_dance.xlsx", index = False)


In [ ]:
# Dancehall
category_id = "0JQ5DAqbMKFP9UNkavIGrM"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dancehall = get_all_playlist_track_info(playlist_ids)
df_dancehall.to_excel("df_dancehall.xlsx", index = False)


In [ ]:
# Decennies
category_id = "0JQ5DAqbMKFIVNxQgRNSg0"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_decennies = get_all_playlist_track_info(playlist_ids)
df_decennies.to_excel("df_decennies.xlsx", index = False)


In [ ]:
# Detente
category_id = "0JQ5DAqbMKFFzDl7qN9Apr"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_detente = get_all_playlist_track_info(playlist_ids)
df_detente.to_excel("df_detente.xlsx", index = False)


In [ ]:
# Dormir
category_id = "0JQ5DAqbMKFCuoRTxhYWow"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dormir = get_all_playlist_track_info(playlist_ids)
df_dormir.to_excel("df_dormir.xlsx", index = False)


In [ ]:
# Enfants
category_id = "0JQ5DAqbMKFFoimhOqWzLB"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_enfants = get_all_playlist_track_info(playlist_ids)
df_enfants.to_excel("df_enfants.xlsx", index = False)


In [ ]:
# En voiture
category_id = "0JQ5DAqbMKFIRybaNTYXXy" 

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voiture = get_all_playlist_track_info(playlist_ids)
df_voiture.to_excel("df_voiture.xlsx", index = False)


In [ ]:
# Equal
category_id = "0JQ5DAqbMKFPw634sFwguI"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_equal = get_all_playlist_track_info(playlist_ids)
df_equal.to_excel("df_equal.xlsx", index = False)


In [ ]:
# Etudiants
category_id = "0JQ5DAqbMKFJw7QLnM27p6"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_etudiants = get_all_playlist_track_info(playlist_ids)
df_etudiants.to_excel("df_etudiants.xlsx", index = False)


In [ ]:
# Focus
category_id = "0JQ5DAqbMKFCbimwdOYlsl"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_focus = get_all_playlist_track_info(playlist_ids)
df_focus.to_excel("df_focus.xlsx", index = False)


In [ ]:
# Folk
category_id = "0JQ5DAqbMKFy78wprEpAjl"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_folk = get_all_playlist_track_info(playlist_ids)
df_folk.to_excel("df_folk.xlsx", index = False)


In [ ]:
# Frequency
category_id = "0JQ5DAqbMKFF9bY76LXmfI"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_frequency = get_all_playlist_track_info(playlist_ids)
df_frequency.to_excel("df_frequency.xlsx", index = False)


In [ ]:
# Fresh Finds
category_id = "0JQ5DAqbMKFImHYGo3eTSg"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_freshfinds = get_all_playlist_track_info(playlist_ids)
df_freshfinds.to_excel("df_freshfinds.xlsx", index = False)


In [ ]:
# Funk
category_id = "0JQ5DAqbMKFFsW9N8maB6z"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_funk = get_all_playlist_track_info(playlist_ids)
df_funk.to_excel("df_funk.xlsx", index = False)


In [ ]:
# Gaming
category_id = "0JQ5DAqbMKFCfObibaOZbv"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_gaming = get_all_playlist_track_info(playlist_ids)
df_gaming.to_excel("df_gaming.xlsx", index = False)


In [ ]:
# Glow
category_id = "0JQ5DAqbMKFGnsSfvg90Wo"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_glow = get_all_playlist_track_info(playlist_ids)
df_glow.to_excel("df_glow.xlsx", index = False)


In [ ]:
# Hip-Hop
category_id = "0JQ5DAqbMKFQ00XGBls6ym"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_hiphop = get_all_playlist_track_info(playlist_ids)
df_hiphop.to_excel("df_hiphop.xlsx", index = False)


In [ ]:
# Indie
category_id = "0JQ5DAqbMKFCWjUTdzaG0e"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_indie = get_all_playlist_track_info(playlist_ids)
df_indie.to_excel("df_indie.xlsx", index = False)


In [ ]:
# Influenceurs
category_id = "0JQ5DAqbMKFRKBHIxJ5hMm"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_influenceurs = get_all_playlist_track_info(playlist_ids)
df_influenceurs.to_excel("df_influenceurs.xlsx", index = False)


In [ ]:
# Jazz
category_id = "0JQ5DAqbMKFAJ5xb0fwo9m"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_jazz = get_all_playlist_track_info(playlist_ids)
df_jazz.to_excel("df_jazz.xlsx", index = False)


In [ ]:
# K-pop
category_id = "0JQ5DAqbMKFGvOw3O4nLAf"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_kpop = get_all_playlist_track_info(playlist_ids)
df_kpop.to_excel("df_kpop.xlsx", index = False)


In [ ]:
# Latino
category_id = "0JQ5DAqbMKFxXaXKP7zcDp"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_latino = get_all_playlist_track_info(playlist_ids)
df_latino.to_excel("df_latino.xlsx", index = False)


In [ ]:
# Metal
category_id = "0JQ5DAqbMKFDkd668ypn6O"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_metal = get_all_playlist_track_info(playlist_ids)
df_metal.to_excel("df_metal.xlsx", index = False)


In [ ]:
# Music instrumentale
category_id = "0JQ5DAqbMKFRieVZLLoo9m" 

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_instrumental = get_all_playlist_track_info(playlist_ids)
df_instrumental.to_excel("df_instrumental.xlsx", index = False)


In [ ]:
# Nature et bruits
category_id = "0JQ5DAqbMKFI3pNLtYMD9S"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_nature = get_all_playlist_track_info(playlist_ids)
df_nature.to_excel("df_nature.xlsx", index = False)


In [ ]:
# Netflix
category_id = "0JQ5DAqbMKFEOEBCABAxo9"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_netflix = get_all_playlist_track_info(playlist_ids)
df_netflix.to_excel("df_netflix.xlsx", index = False)


In [ ]:
# Pop
category_id = "0JQ5DAqbMKFEC4WFtoNRpw"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_pop = get_all_playlist_track_info(playlist_ids)
df_pop.to_excel("df_pop.xlsx", index = False)


In [ ]:
# Punk
category_id = "0JQ5DAqbMKFAjfauKLOZiv"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_punk = get_all_playlist_track_info(playlist_ids)
df_punk.to_excel("df_punk.xlsx", index = False)


In [5]:
# Radar
category_id = "0JQ5DAqbMKFOOxftoKZxod"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_radar = get_all_playlist_track_info(playlist_ids)
df_radar.to_excel("df_radar.xlsx", index = False)


Processing playlist: 37i9dQZF1DWTU63finroJM
Processing playlist: 37i9dQZF1DWY0DyDKedRYY
Processing playlist: 37i9dQZF1DWYc8Z858eGwP
Processing playlist: 37i9dQZF1DX5WmphwNfZfk
Processing playlist: 37i9dQZF1DWV7gN1qBoBYg
Processing playlist: 37i9dQZF1DX9IALXsyt8zk
Processing playlist: 37i9dQZF1DWVp4cWdnm5ws
Processing playlist: 37i9dQZF1DWTVE8ZCcLQ5Q
Processing playlist: 37i9dQZF1DX0dUsjwqfaGM
Processing playlist: 37i9dQZF1DX2P3E6UOxZyt
Processing playlist: 37i9dQZF1DX4OR8pnFkwhR


In [6]:
# Reggae
category_id = "0JQ5DAqbMKFJKoGyUMo2hE"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_reggae = get_all_playlist_track_info(playlist_ids)
df_reggae.to_excel("df_reggae.xlsx", index = False)


Processing playlist: 37i9dQZF1DX2oc5aN4UDfD
Processing playlist: 37i9dQZF1DXa8n42306eJB
Processing playlist: 37i9dQZF1DXbwoaqxaoAVr
Processing playlist: 37i9dQZF1DX3fXJqxGjuEP
Processing playlist: 37i9dQZF1DX8cLSL1n19DR


In [7]:
# RnB
category_id = "0JQ5DAqbMKFEZPnFQSFB1T"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rnb = get_all_playlist_track_info(playlist_ids)
df_rnb.to_excel("df_rnb.xlsx", index = False)


Processing playlist: 37i9dQZF1DX2LoIIQLAhdb
Processing playlist: 37i9dQZF1DX9UuQbl12Nmb
Processing playlist: 37i9dQZF1DXch2nNxu927l
Processing playlist: 37i9dQZF1DWYmmr74INQlb
Processing playlist: 37i9dQZF1DWXnexX7CktaI
Processing playlist: 37i9dQZF1DWXbttAJcbphz
Processing playlist: 37i9dQZF1DX2PG4mbkilf3
Processing playlist: 37i9dQZF1DX942d1NzGgzk
Processing playlist: 37i9dQZF1DX2MJVTOdWtbm
Processing playlist: 37i9dQZF1DWUbo613Z2iWO
Processing playlist: 37i9dQZF1DX2V97dgq1s0W
Processing playlist: 37i9dQZF1DWY90iCJMKual


In [ ]:
# Rock
category_id = "0JQ5DAqbMKFDXXwE9BDJAr"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rock = get_all_playlist_track_info(playlist_ids)
df_rock.to_excel("df_rock.xlsx", index = False)


Processing playlist: 37i9dQZF1DWWSuZL7uNdVA
Processing playlist: 37i9dQZF1DWTBN71pVn2Ej
Processing playlist: 37i9dQZF1DWZryfp6NSvtz
Processing playlist: 37i9dQZF1DWXTHBOfJ8aI7
Processing playlist: 37i9dQZF1DX8HxJlVUR3HN
Processing playlist: 37i9dQZF1DXcUF9GRO45to
Processing playlist: 37i9dQZF1DX2xKCIjknUoQ
Processing playlist: 37i9dQZF1DXbJR1DWfHjWw
Processing playlist: 37i9dQZF1DX99DRG9N39X3
Processing playlist: 37i9dQZF1DX9xdWyDn7XX8
Processing playlist: 37i9dQZF1DX89XXHpIgTCJ
Processing playlist: 37i9dQZF1DX35DWKgAk2B5
Processing playlist: 37i9dQZF1DX6L1CUnSfVed
Processing playlist: 37i9dQZF1DX1rVvRgjX59F
Processing playlist: 37i9dQZF1DX3oM43CtKnRV
Processing playlist: 37i9dQZF1DX11ghcIxjcjE
Processing playlist: 37i9dQZF1DWVqJMsgEN0F4
Processing playlist: 37i9dQZF1DWT2jS7NwYPVI
Processing playlist: 37i9dQZF1DWWJOmJ7nRx0C
Processing playlist: 37i9dQZF1DX78toxP7mOaJ
Processing playlist: 37i9dQZF1DX2UXfvEIZvDK
Processing playlist: 37i9dQZF1DX8FwnYE6PRvL
Processing playlist: 37i9dQZF1DW

In [9]:
# Soirée
category_id = "0JQ5DAqbMKFA6SOHvT3gck"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soiree = get_all_playlist_track_info(playlist_ids)
df_soiree.to_excel("df_soiree.xlsx", index = False)


Processing playlist: 37i9dQZF1DWUwJ0RFwrgQP
Processing playlist: 37i9dQZF1DWZEbxox96IUk
Processing playlist: 37i9dQZF1DXbIqU3eEXwop
Processing playlist: 37i9dQZF1DWWylYLMvjuRG
Processing playlist: 37i9dQZF1DXdo6A3mWpdWx
Processing playlist: 37i9dQZF1DX1Hya1sRqqxI
Processing playlist: 37i9dQZF1DX3AdAEX3vkB1
Processing playlist: 37i9dQZF1DX9pP7cC2liKw
Processing playlist: 37i9dQZF1DX2pprEpa9URZ
Processing playlist: 37i9dQZF1DXcrDJ7TeNU58
Processing playlist: 37i9dQZF1DX2KwEtNSejGe
Processing playlist: 37i9dQZF1DX0pH2SQMRXnC
Processing playlist: 37i9dQZF1DX1ZtBovtTdHT
Processing playlist: 37i9dQZF1DX8a1tdzq5tbM
Processing playlist: 37i9dQZF1DWXti3N4Wp5xy
Processing playlist: 37i9dQZF1DWZQaaqNMbbXa
Processing playlist: 37i9dQZF1DX0Uv9tZ47pWo
Processing playlist: 37i9dQZF1DWVxqA801UP5W
Processing playlist: 37i9dQZF1DXcKW8VlJappa
Processing playlist: 37i9dQZF1DWY4xHQp97fN6
Processing playlist: 37i9dQZF1DX62HvDmsP7Md
Processing playlist: 37i9dQZF1DWTAZ0XePuYqW
Processing playlist: 37i9dQZF1DX

In [10]:
# Soul
category_id = "0JQ5DAqbMKFIpEuaCnimBj"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soul = get_all_playlist_track_info(playlist_ids)
df_soul.to_excel("df_soul.xlsx", index = False)


Processing playlist: 37i9dQZF1DX9XIFQuFvzM4
Processing playlist: 37i9dQZF1DWULEW2RfoSCi
Processing playlist: 37i9dQZF1DXbcgQ8d7s0A0
Processing playlist: 37i9dQZF1DWTx0xog3gN3q
Processing playlist: 37i9dQZF1DXchlyaSeZp0q
Processing playlist: 37i9dQZF1DX3ohNxI5tB79
Processing playlist: 37i9dQZF1DWYmZJhCzQOPD
Processing playlist: 37i9dQZF1DX0H8hDpv38Ju
Processing playlist: 37i9dQZF1DX8Md3JnnrexB
Processing playlist: 37i9dQZF1DX8MbMfAHb8U0


In [11]:
# Sport
category_id = "0JQ5DAqbMKFAXlCG6QvYQ4"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_sport = get_all_playlist_track_info(playlist_ids)
df_sport.to_excel("df_sport.xlsx", index = False)


Processing playlist: 37i9dQZF1DX4osfY3zybD2
Processing playlist: 37i9dQZF1DX76t638V6CA8
Processing playlist: 37i9dQZF1DX76Wlfdnj7AP
Processing playlist: 37i9dQZF1DWVhQ5d3I6DeF
Processing playlist: 37i9dQZF1DXdxcBWuJkbcy
Processing playlist: 37i9dQZF1DXadOVCgGhS7j
Processing playlist: 37i9dQZF1DWUxdwkOJZYCJ
Processing playlist: 37i9dQZF1DX0hWmn8d5pRe
Processing playlist: 37i9dQZF1DWZUTt0fNaCPB
Processing playlist: 37i9dQZF1DWWPcvnOpPG3x
Processing playlist: 37i9dQZF1DWZq91oLsHZvy
Processing playlist: 37i9dQZF1DX9oh43oAzkyx
Processing playlist: 37i9dQZF1DX0HRj9P7NxeE
Processing playlist: 37i9dQZF1DX5n5gZBZb0AT
Processing playlist: 37i9dQZF1DWTl4y3vgJOXW
Processing playlist: 37i9dQZF1DWSRbZUjHUFIA


In [12]:
# Spotify singles
category_id = "0JQ5DAqbMKFDBgllo2cUIN"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_spotsingles = get_all_playlist_track_info(playlist_ids)
df_spotsingles.to_excel("df_spotsingles.xlsx", index = False)


Processing playlist: 37i9dQZF1DWZqyWCBK6QTt
Processing playlist: 37i9dQZF1DWTUm9HjVUKnL
Processing playlist: 37i9dQZF1DWVP71DYelp29
Processing playlist: 37i9dQZF1DXdam5VrqMf3y
Processing playlist: 37i9dQZF1DXdFhiq9L31be
Processing playlist: 37i9dQZF1DWXMetQrUYhar
Processing playlist: 37i9dQZF1DXdXh4ecan7Hh
Processing playlist: 37i9dQZF1DXcaEXdyjLdxm


In [13]:
# Tendances
category_id = "0JQ5DAqbMKFQIL0AXnG5AK"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tendances = get_all_playlist_track_info(playlist_ids)
df_tendances.to_excel("df_tendances.xlsx", index = False)


Processing playlist: 37i9dQZF1DWWUZ614R8EUb


In [14]:
# TV et cinéma
category_id = "0JQ5DAqbMKFOzQeOmemkuw"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tvcine = get_all_playlist_track_info(playlist_ids)
df_tvcine.to_excel("df_tvcine.xlsx", index = False)


Processing playlist: 37i9dQZF1DXbLzW15wHm9R
Processing playlist: 37i9dQZF1DXb69UWhjrXsW
Processing playlist: 37i9dQZF1DX6dCTsdzLHOw
Processing playlist: 37i9dQZF1DX6K3W8KBiALe
Processing playlist: 37i9dQZF1DX4Jyx1uIhxnP
Processing playlist: 37i9dQZF1DXar0WmW5YgAc
Processing playlist: 37i9dQZF1DX0ZzMQgXnSUD
Processing playlist: 37i9dQZF1DXd4bJEFQJTXh


In [15]:
# Variété
category_id = "0JQ5DAqbMKFBweu2euf3Sj"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_variete = get_all_playlist_track_info(playlist_ids)
df_variete.to_excel("df_variete.xlsx", index = False)


Processing playlist: 37i9dQZF1DWTIykNHRogOx
Processing playlist: 37i9dQZF1DX9h4FeDa1xsw
Processing playlist: 37i9dQZF1DWZafi8zKNyuo
Processing playlist: 37i9dQZF1DX2vfncMeBctk
Processing playlist: 37i9dQZF1DWSZsRTzm4O6a
Processing playlist: 37i9dQZF1DWTypZHlgEy1G
Processing playlist: 37i9dQZF1DX8OVycSP3AXz
Processing playlist: 37i9dQZF1DX2clypTgBmEw


In [16]:
# Voyage
category_id = "0JQ5DAqbMKFAQy4HL4XU2D"

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voyage = get_all_playlist_track_info(playlist_ids)
df_voyage.to_excel("df_voyage.xlsx", index = False)


Processing playlist: 37i9dQZF1DX1I7noGgAP1O
Processing playlist: 37i9dQZF1DXaYIs1DV1CwP
Processing playlist: 37i9dQZF1DX9ZaBESVF0Yv
Processing playlist: 37i9dQZF1DWXjv2J1ebzlh
Processing playlist: 37i9dQZF1DWUoqEG4WY6ce
Processing playlist: 37i9dQZF1DWYRp9Xqj0c83
Processing playlist: 37i9dQZF1DWYhUoxbsQ1Ul
